In [1]:
!pip install PyMuPDF
!pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 10.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-abvj4rzw
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-abvj4rzw
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=0b6f9e4904b05feb0fb083914408cbe7977c9160db644f3e47d571667c2ff586
  Stored in directory: /tmp/pip-ephem-wheel-cache-lbgifnn1/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import re

import tensorflow as tf
import matplotlib.pyplot as plt

from nltk import word_tokenize

In [4]:
import fitz

path_output = '/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Test_Model/CV_test.pdf'
with open(r'/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Test_Model/CV_test.txt', 'w', encoding='utf-8') as f:
    with fitz.open(path_output) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
            text = re.sub(' *| *| *|• *| *| *| *|ü *|● *|� *| *| *|▪ *|\s+-', ' ', text)
            text = re.sub('\n\s+', '\n', text)
        print('-------------------------------------------------------------------------------------')
        print(text)
        print('-------------------------------------------------------------------------------------')
        f.write(text)

-------------------------------------------------------------------------------------
Le Minh Binh 
University of Science Ho Chi Minh city 
Address: 333 Ben Binh Dong Street, Ward 11, District 8, HCM City 
Phone: +84 1653417308| Email: binhleminh0209@gmail.com 
Date of Birth: September 2nd, 1996 
CAREER OBJECTIVE: 
Seeking a dynamic studying and working environment in the field of Machine 
Learning and Data Science that can improve my skills and exploit my abilities. 
TECHNICAL SKILLS: 
Programming language: Python (with frameworks: tensorflow, sklearn, numpy, 
pandas), C/C++, Matlab, R….  
Mathematical problem solving 
Microsoft Office (MS Word, Excel, PowerPoint) 
PERSONAL SKILLS: 
Ability to work independently or as part of a team 
Ability to work under pressure 
Verbal communication skill 
EDUCATION: 
Senior student of Mathematical Computer Science (2015-2018) 
Major: Mathematical Computing.  
Cumulative GPA: 3.66/4.0 
Faculty of Mathematics and Computer Science, University of Scie

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load('/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Build_Vocabulary/word2vec_v8.bin', mmap='r')

In [7]:
def decontracted(phrase):
    # specific
    phrase = re.sub(":|-\s+|,|–\s+|-|\||\/", " ", phrase)
    phrase = re.sub(r"â€|™|“|”|;|ü|\xad|\xa0|\u200b|·|∙|�|●|�|§|•|!|▪|©|\?|\]|\[|\)|\(", "", phrase)
    phrase = re.sub(r"[^\w\s]", " ", phrase)
    
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    
    phrase = re.sub(r"\bhcmc\b", "ho chi minh city ", phrase)
    phrase = re.sub(r"\bhcm\b", "ho chi minh ", phrase)
    phrase = re.sub(r"\bbs\b ", "bachelor of ", phrase)
    phrase = re.sub(r"r’s", "r of", phrase)
    phrase = re.sub(r"\’s", " of", phrase)
    
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " of", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\bim\b", "i am ", phrase)
    phrase = re.sub(r"\bi’m\b", "i am ", phrase)
    
    return phrase

def sent2idx(x):
    xidx = []
    for w in word_tokenize(x):
        if w in wv.key_to_index:
            xidx.append(wv.key_to_index[w])
    return np.array(xidx)

In [8]:
from keras.models import load_model

from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy

model = load_model('/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Build_Model_Segment/bilstm_crf_v4.h5', custom_objects={'CRF': CRF, 'crf_loss': crf_loss , 
                                                                                                                       'crf_viterbi_accuracy': crf_viterbi_accuracy})

In [9]:
# tag for model version 4
tag2idx = {'I-W': 0, 'nan': 1, 'B-SU': 2, 'I-S': 3, 'B-I': 4, 'I_I': 5, 'B-S': 6, 'I-I': 7, 'I-E': 8, 'I-SU': 9, 'B-E': 10, 'B-Su': 11, 'O': 12, 'B-W': 13} 
idx2tag = {0: 'I-W', 1: 'nan', 2: 'B-SU', 3: 'I-S', 4: 'B-I', 5: 'I_I', 6: 'B-S', 7: 'I-I', 8: 'I-E', 9: 'I-SU', 10: 'B-E', 11: 'B-Su', 12: 'O', 13: 'B-W'}

# n_tags = len(tag2idx)
# print(tag2idx, '\nn_tag:' ,n_tags)

In [10]:
test_line = []

nfile = '/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Test_Model/CV_test.txt'
with open(nfile, 'r', encoding = 'utf-8') as f:
    raw = f.readlines()
    for i in raw:
        clean = decontracted(str(i).lower())
        test_line.append(clean.strip())
    
test_data = []
for i in test_line:
    test_data.append(sent2idx(i))

for i, x in enumerate(test_data):
    if len(x) < 64:
        test_data[i] = np.hstack((x, np.zeros(64-len(x))))


#p = model.predict(np.array([test_data[i]]))
#p = np.argmax(p, axis=-1)
# Visualization
print("{:90}||{:5}".format("sentences", "pre tag"))
print(100 * "=")
lst_skill = []
lst_info = []
lst_edu = []
lst_exp = []
lst_obj = []
for i, w in zip(test_data, test_line):
    p = model.predict(np.array([i]))
    p = np.argmax(p)
    print("{:90}: {:5}".format(w, idx2tag[p]))

    if idx2tag[p]=='B-I' or idx2tag[p]=='I-I':
        lst_info.append(w)
    if idx2tag[p]=='B-S' or idx2tag[p]=='I-S':
        lst_skill.append(w)
    if idx2tag[p]=='B-E' or idx2tag[p]=='I-E':
        lst_edu.append(w)
    if idx2tag[p]=='B-W' or idx2tag[p]=='I-W':
        lst_exp.append(w)
    if idx2tag[p]=='B-SU' or idx2tag[p]=='I-SU':
        lst_obj.append(w)

text1 = ', '.join(lst_info)
text2 = ', '.join(lst_skill)
text3 = ', '.join(lst_edu)
text4 = ', '.join(lst_exp)
text5 = ', '.join(lst_obj)

info = re.sub('\s+', ' ', str(text1))
skill = re.sub('\s+', ' ', str(text2))
edu = re.sub('\s+', ' ', str(text3))
exp = re.sub('\s+', ' ', str(text4))
obj = re.sub('\s+', ' ', str(text5))

sentences                                                                                 ||pre tag
le minh binh                                                                              : B-I  
university of science ho chi minh city                                                    : I-E  
address  333 ben binh dong street  ward 11  district 8  ho chi minh  city                 : I-I  
phone   84 1653417308  email  binhleminh0209 gmail com                                    : I-I  
date of birth  september 2nd  1996                                                        : I-I  
career objective                                                                          : B-SU 
seeking a dynamic studying and working environment in the field of machine                : I-SU 
learning and data science that can improve my skills and exploit my abilities             : I-SU 
technical skills                                                                          : B-S  
programming langua

In [11]:
print('1.Information: ', info)
print('2.Skill: ', skill)
print('3.Education: ', edu)
print('4.Experience: ', exp)

1.Information:  le minh binh, address 333 ben binh dong street ward 11 district 8 ho chi minh city, phone 84 1653417308 email binhleminh0209 gmail com, date of birth september 2nd 1996, ward 4 district 5 ho chi minh city
2.Skill:  technical skills, programming language python with frameworks tensorflow sklearn numpy, pandas c c matlab r, mathematical problem solving, microsoft office ms word excel powerpoint, ability to work independently or as part of a team, ability to work under pressure, verbal communication skill
3.Education:  university of science ho chi minh city, education, senior student of mathematical computer science 2015 2018, major mathematical computing, cumulative gpa 3 66 4 0, faculty of mathematics and computer science university of science ho chi minh city, exchanged student faculty of engineering
4.Experience:  personal skills, achievements responsibilities, pre professional experience, being a tutor 2015 2017, being a salesman for a retail agent june 2016 august 20

In [12]:
extracted_data1 = pd.DataFrame({'Extract_Information': [info],
                               'Extract_Skill': [skill],
                              'Extract_Education': [edu],
                               'Extract_Experience': [exp],
                              })

pd.set_option("display.max_colwidth", -1)
extracted_data1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys



,Extract_Information,Extract_Skill,Extract_Education,Extract_Experience
0,"le minh binh, address 333 ben binh dong street ward 11 district 8 ho chi minh city, phone 84 1653417308 email binhleminh0209 gmail com, date of birth september 2nd 1996, ward 4 district 5 ho chi minh city","technical skills, programming language python with frameworks tensorflow sklearn numpy, pandas c c matlab r, mathematical problem solving, microsoft office ms word excel powerpoint, ability to work independently or as part of a team, ability to work under pressure, verbal communication skill","university of science ho chi minh city, education, senior student of mathematical computer science 2015 2018, major mathematical computing, cumulative gpa 3 66 4 0, faculty of mathematics and computer science university of science ho chi minh city, exchanged student faculty of engineering","personal skills, achievements responsibilities, pre professional experience, being a tutor 2015 2017, being a salesman for a retail agent june 2016 august 2016, tokyo university of agriculture and technology june 2017 august 2017, koganei tokyo japan, handle problems related to facial emotion recognition using kinect toolbox"


In [13]:
def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"', " ", str(input_text))
    
    word_tokens = nltk.tokenize.word_tokenize(word_tokens)
    
    
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    # create a set to keep the results in.
    found_skills = set()

    # search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)

    # search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)

    return found_skills

In [14]:
#Extract Skills
skills = pd.read_csv('/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Data/skills_data.csv', encoding='latin')
SKILLS_DB = list(set(skills['Skills'].values))

df_skill = pd.DataFrame(columns=['extract_skill'])
skills1 = extract_skills(skill)
skills2 = extract_skills(exp)
    
# z = str(i) + ''.join(str(j))
join_skills = skills1.union(skills2)
df_skill = df_skill.append({'extract_skill': join_skills}, ignore_index=True)
#     df_skill = df_skill.append({'skill': z, 'extract_skill': join_skills}, ignore_index=True)


#Extract Education
edu_pattern = re.compile('college|master|bachelor|bachelors|.*major.*')
df_edu = pd.DataFrame(columns=['extract_edu'])

lst_edu = []
lst = str(edu).split(',')
for j in lst:
      edu = edu_pattern.findall(j)
      if bool(edu) == True:
          lst_edu.extend(edu)
        
df_edu = df_edu.append({'extract_edu': lst_edu}, ignore_index=True)
#     df_edu = df_edu.append({'education': i, 'extract_edu': lst_edu}, ignore_index=True)


#Extract Experience
year_pattern = re.compile('([2]{1}[0]{1}[0-9]{2})')
exp_pattern = re.compile('([0-9]{2})\s+year')

df_exp = pd.DataFrame(columns=['experience', 'extract_exp'])
lst_exp = []
text = str(edu) + ' ' + str(exp)
exp = exp_pattern.findall(text)
years = year_pattern.findall(text)
    
if bool(exp)==True:
    to_int = [int(i) for i in exp]
    extract_year = exp
    n_exp = max(to_int)
else:
    to_int = [int(i) for i in years]
    extract_year = years
    if len(to_int) > 0:
        max_exp = max(to_int)
        min_exp = min(to_int)
        n_exp = max_exp - min_exp
    else:
        n_exp = 'None'
            
lst_exp.append(str(n_exp))
df_exp = df_exp.append({'extract_exp': lst_exp}, ignore_index=True)


In [15]:
extracted_data = pd.DataFrame({'Extract_Skill': df_skill['extract_skill'],
                               'Extract_Edu': df_edu['extract_edu'],
                              'Extract_Exp': df_exp['extract_exp'],
                              })

pd.set_option("display.max_colwidth", -1)
extracted_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  



,Extract_Skill,Extract_Edu,Extract_Exp
0,"{pandas, tensorflow, numpy, c, python, office, word, r, microsoft office, mathematical problem solving, excel, problem solving, agriculture, powerpoint, matlab}",[ major mathematical computing],[2]
